In [10]:
# standard libraries
import csv
import time
import numpy as np
import pandas as pd
import re
import gc
from collections import defaultdict

# data visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import sklearn as sk  
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression  
from sklearn import svm  
from sklearn.ensemble import RandomForestClassifier  
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df_train = pd.read_csv('train.tsv', delimiter='\t', index_col=['train_id'])
df_test = pd.read_csv('test.tsv', delimiter='\t', index_col=['test_id'])

df_train.head(5)

C:\Users\Alice\Anaconda2\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,name,item_condition_id,category_name,brand_name,price,shipping,item_description
train_id,,,,,,,
0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [3]:
df_test.head(5)

,name,item_condition_id,category_name,brand_name,shipping,item_description
test_id,,,,,,
0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


In [4]:
brands = df_train.groupby(['brand_name']).size()

print "There are unique brands:", brands.shape[0]
print "of which {} have no brand assigned".format(df_train[df_train['brand_name'].isnull()].shape[0])

brands.sort_values(ascending=False).head(20)

There are unique brands: 4809
of which 632682 have no brand assigned


brand_name
PINK                 54088
Nike                 54043
Victoria's Secret    48036
LuLaRoe              31024
Apple                17322
FOREVER 21           15186
Nintendo             15007
Lululemon            14558
Michael Kors         13928
American Eagle       13254
Rae Dunn             12305
Sephora              12172
Coach                10463
Disney               10360
Bath & Body Works    10354
Adidas               10202
Funko                 9237
Under Armour          8461
Sony                  7994
Old Navy              7567
dtype: int64

In [5]:
#vectorize the brand name for training data

le = preprocessing.LabelEncoder()
X_tr = le.fit_transform(df_train['brand_name'])
X_tr=np.array(X_tr)
#X_tr=X_tr.reshape(-1, 1)

print df_train.shape
print X_tr.shape

(1482535, 7)
(1482535L,)


In [6]:
#make three classes for price
def priceClass(p):
    if p < 50:
        p_class=0
    elif p>50 and p<100:
        p_class = 1
    else:
        p_class=2
        
    return p_class   

price_class = []
for p in df_train['price']:
    price_class.append(priceClass(p)) 

y_tr=np.array(price_class)

df_test.head()

print df_train['price'].shape
print y_tr.shape
X_tr, X_val, y_tr, y_val = train_test_split(X_tr, y_tr, test_size=0.25)


(1482535L,)
(1482535L,)


In [7]:
X_tr=X_tr.reshape(-1, 1)
X_val=X_val.reshape(-1, 1)

In [ ]:
#training SVM model, takes a long  time
#clf = svm.SVC(gamma='auto')
#clf.fit(X_tr, y_tr) 
#y_pred=clf.predict(X_val)

In [8]:
k=3
neigh = KNeighborsClassifier(n_neighbors=k)
neigh.fit(X_tr, y_tr)
y_pred=neigh.predict(X_val)

In [11]:
print "accuracy score: ",accuracy_score(y_val, y_pred)
print '\n'
print 'confusion matrix: \n',confusion_matrix(y_val,y_pred)
print '\n'
print(classification_report(y_val, y_pred))

accuracy score:  0.885814577184


confusion matrix: 
[[323849   3959   2782]
 [ 22592   2407   1564]
 [ 10205   1219   2057]]


             precision    recall  f1-score   support

          0       0.91      0.98      0.94    330590
          1       0.32      0.09      0.14     26563
          2       0.32      0.15      0.21     13481

avg / total       0.84      0.89      0.86    370634



In [ ]:
#optional logistic regression
#LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_tr, y_tr)

In [17]:
#mlp = MLPClassifier(hidden_layer_sizes=(20, 20, 20), max_iter=5000)  
mlp=MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(2, 2), random_state=1)
mlp.fit(X_tr, y_tr) 
y_predi = mlp.predict(X_val) 

print "accuracy score: ",accuracy_score(y_val, y_predi)
print '\n'
print 'confusion matrix: \n',confusion_matrix(y_val,y_predi)
print '\n'
print(classification_report(y_val, y_predi))

accuracy score:  0.891958104221


confusion matrix: 
[[330590      0      0]
 [ 26563      0      0]
 [ 13481      0      0]]


             precision    recall  f1-score   support

          0       0.89      1.00      0.94    330590
          1       0.00      0.00      0.00     26563
          2       0.00      0.00      0.00     13481

avg / total       0.80      0.89      0.84    370634



C:\Users\Alice\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
